In [10]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [ ]:
import logging
from pathlib import Path

import altair as alt
import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.labels import add_labels
from src.features.scaling import scale_min_max
from src.features.transforming import interpolate_and_fill_nulls, map_trials, merge_dfs
from src.features.utils import to_describe
from src.log_config import configure_logging
from src.plots.confidence_intervals import plot_confidence_intervals
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
)
from src.plots.utils import prepare_multiline_hvplot

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [12]:
db = DatabaseManager()

In [21]:
with db:
    pupil = db.get_table("Feature_Pupil")
    eda = db.get_table("Feature_EDA")
    stimulus = db.get_table("Feature_Stimulus")
    trials = db.get_table("Trials")


df = merge_dfs([pupil, stimulus, eda])
df = merge_dfs(
    dfs=[df, trials],
    on=["trial_id", "participant_id", "trial_number"],
)
df = interpolate_and_fill_nulls(df)
df = df.with_columns(col("pupil_mean").alias("pupil"))

In [ ]:
col1, col2 = "pupil", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
# plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [ ]:
col1, col2 = "pupil", "temperature"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
# plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [ ]:
col1, col2 = "eda_tonic", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)